In [10]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict ,Annotated
from pydantic import BaseModel,Field
import operator
from langchain_core.runnables.graph import MermaidDrawMethod
print(" imports OK")

 imports OK


In [11]:
load_dotenv()

True

In [12]:
model =ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [13]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0 , le=10)

In [14]:
structured_model = model.with_structured_output(EvaluationSchema)

In [15]:
essay_prompt = """ Mental health is an essential part of overall well-being. Just as physical health keeps our body strong, mental health ensures that our mind functions in a balanced and positive way. It refers to our emotional, psychological, and social well-being, influencing how we think, feel, and behave in daily life. A healthy mind helps us handle stress, build relationships, and make wise decisions. Unfortunately, mental health is often overlooked, even though it is as important as physical health.

Good mental health allows people to cope with challenges, enjoy life, and contribute positively to society. Students with strong mental health can focus better on their studies. Adults with balanced mental health can perform effectively at work and maintain healthy relationships. Moreover, communities where mental health is valued are more productive, peaceful, and supportive.

Several factors can harm mental well-being, such as stress, trauma, poverty, family problems, or health conditions. Modern lifestyles, marked by constant pressure, competition, and social media, often increase feelings of anxiety, loneliness, and depression. Unfortunately, stigma and lack of awareness prevent many people from seeking help. This can worsen their condition and even lead to serious consequences like self-harm or suicide.

Maintaining mental health requires conscious effort. Some effective ways include open conversations about feelings, adopting a healthy lifestyle with balanced food, exercise, and proper sleep, and managing stress through meditation or relaxation techniques. Seeking professional help from psychologists or counselors when needed is equally important, as is building strong relationships with family and friends for emotional support.

Mental health is not a luxury; it is a necessity for a happy and successful life. Just as we visit doctors for physical illnesses, we must also seek help for mental struggles without shame. Governments, schools, workplaces, and families should work together to spread awareness and provide resources. When mental health is prioritized, individuals and societies become stronger, more resilient, and more compassionate."""

In [18]:
prompt= f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n (essay)"

structured_model.invoke(prompt).feedback

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [19]:
class UPSCState(TypedDict):
    eassy :str
    langguage_feedback :str
    analysis_feedback :str
    clarity_feedback :str
    overall_feedback : str
    individual_scores :Annotated[list[int],operator.add]
    avg_score :float

In [76]:
def evaluate_language(state:UPSCState):
    prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n (essay){state.essay}"
    output = structured_model.invoke(prompt)

    return {'language_feedback':output.feedback ,'individual_scores':[output.score]}

In [77]:
def evaluate_analysis(state:UPSCState):
    prompt = f"Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n (essay){state.essay}"
    output = structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback ,'individual_scores':[output.score]}

In [78]:
def evaluate_thought(state:UPSCState):
    prompt = f"Evaluate the clarity of thoughtof the following essay and provide a feedback and assign a score out of 10 \n (essay){state.essay}"
    output = structured_model.invoke(prompt)

    return {'thought_feedback':output.feedback ,'individual_scores':[output.score]}

In [79]:
def evaluate_language(state:UPSCState):
    prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n (essay){state.essay}"
    output = structured_model.invoke(prompt)

    return {'language_feedback':output.feedback ,'individual_scores':[output.score]}

In [80]:
def final_evaluation(state:UPSCState):
    
    # Summary feedback
    prompt = (
        f'Based on the following feedbacks create a summarized feedback:\n'
        f'language feedback - {state["language_feedback"]}\n'
        f'depth of analysis feedback - {state["analysis_feedback"]}\n'
        f'clarity of thought feedback - {state.get("thought_feedback", "")}\n'
        f'clarity of thought feedback - {state.get("clarity_feedback", "")}'
    )
    overall_feedback = model.invoke(prompt).content
    
    # avg calculate
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])
    
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}
    

In [81]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

#edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')


graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()



In [ ]:
workflow

In [ ]:
initial_state = {
    'essay': 'essay'
}

workflow.invoke(initial_state)